In [1]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'sst2_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.sst2', 'data_parameters': {'name': 'sst2', 'train_batch_size': 64, 'test_batch_size': 64}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 12, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.0001, 'weight_decay': 1e-05}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.5}}, 'loss_configs': {'loss_class': 'torch.nn.CrossEntropyLoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 10325, 'n

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [3]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  21290


In [4]:
print('loading dataset...')
data_loader = data_obj.load()

loading dataset...


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...


100%|██████████| 1053/1053 [00:10<00:00, 101.98it/s, epoch=0/12, loss=0.438, lr=0.0001, metric_score=0.667, time=10.4]


Epoch: 0, Test Loss: 0.4846453581537519, Test Score: 0.7775229357798165, Time Cost: 0.1037740707397461


100%|██████████| 1053/1053 [00:09<00:00, 107.48it/s, epoch=1/12, loss=0.139, lr=5e-5, metric_score=0.952, time=20.3]


Epoch: 1, Test Loss: 0.49622033536434174, Test Score: 0.7912844036697247, Time Cost: 0.06569600105285645


...

Epoch: 2, Test Loss: 0.5071662366390228, Test Score: 0.7947247706422018, Time Cost: 0.05881500244140625


100%|██████████| 1053/1053 [00:08<00:00, 118.60it/s, epoch=3/12, loss=0.319, lr=1.25e-5, metric_score=0.905, time=38.5]


Epoch: 3, Test Loss: 0.5115950107574463, Test Score: 0.7970183486238532, Time Cost: 0.06632614135742188


 45%|████▌     | 475/1053 [00:04<00:05, 110.17it/s, epoch=4/12, loss=0.176, lr=6.25e-6, metric_score=0.953, time=42.9]


KeyboardInterrupt: 

In [ ]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

In [ ]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

In [ ]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))